In [42]:

# %aimport rl_envs.grid_world_env

import torch
import math
from torch.utils.tensorboard import SummaryWriter # type: ignore

from rl_envs.gym_grid_world_env import GridWorldEnv
from agents.offline_A2C import A2CAgent
from tools.helper import *
import torch.utils.data
%load_ext autoreload 
%autoreload 2

from stable_baselines3 import A2C
from torch.nn import functional as F
import gymnasium as gym

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
LEARN_RATE = 1e-5
DISCOUNTED_FACTOR = 0.99

FORBIDDEN_REWARD = -10
HITWALL_REWARD = -10
TARGET_REWARD = 1

In [44]:
# env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# # env = GridWorldEnv(size=3,fixed_map = True, forbidden_grids=[(1,1)], target_grids=[(2,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# model = A2C("MultiInputPolicy", env, tensorboard_log="./runs/", verbose=1)
# model.learn(total_timesteps=100_0000) # 所以训练无 fobidden 的地图需要 10_0000 次 (反正 1_0000 是不够的) 
# env.close()

In [45]:
# # vec_env = GridWorldEnv(size=3,fixed_map = True, forbidden_grids=[(1,1)], target_grids=[(2,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# vec_env = GridWorldEnv( fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD, render_mode="human")

# obs, _ = vec_env.reset()
# total_reward = 0
# for i in range(500):
#     action, _states = model.predict(obs, deterministic=True)
#     obs, reward, terminated, truncated, info  = vec_env.step(action)
#     # vec_env.render()
#     # if reward > 0:
#         # break
#     # VecEnv resets automatically
#     total_reward += reward
#     if terminated or truncated:
#         obs, _ = vec_env.reset()
#         print('reward: {}, distance: {}'.format(total_reward, info))
#         total_reward = 0
#         if truncated:
#             print("TRUNCATE")
#         else:
#             print("TERMINATE")
# vec_env.close()

In [46]:
# test_env = GridWorldEnv( fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD, render_mode="human")
# test_obs = {'agent': np.array([0, 0]), 'target': np.array([[3, 2]]), 'forbidden': np.array([[1, 1],
#     [1, 2],
#     [2, 2],
#     [3, 1],
#     [3, 3],
#     [4, 1]])}

# V = {}
                                   
# observation, _ = model.policy.obs_to_tensor(test_obs)
# # obs_as_tensor(agent._last_obs, agent.device)
# actions, values, log_prob = model.policy(observation)
# for y in range(env.size):
#     print("[", end=" ")
#     for x in range(env.size):
#         test_obs['agent'] = np.array([y,x])
#         observation, _ = model.policy.obs_to_tensor(test_obs)
#         actions, state_values, log_prob = model.policy(observation)
#         action = np.argmax(actions)
#         V[(y,x)] = state_values.item()
#         print(test_env.action_mappings[action], end=" ")
#     print("]")

# print_by_dict(test_env, V)

#        # model.policy.forward(test_obs, deterministic=True)
#        # q_values = model.q_net(observation)
#        # Q[(y,x)] = q_values

# ''


In [47]:
# env = GridWorldEnv(size=3,fixed_map = True, forbidden_grids=[(1,1)], target_grids=[(2,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
env = gym.make('CliffWalking-v0')



In [48]:

# torch.autograd.set_detect_anomaly(False, check_nan=True)

# agent.RUN(env)

In [49]:
"""
collect experience replay
"""
episode_len = 1000
batch_size = 100

obs, _ = env.reset()
next_state = torch.tensor(obs, dtype = torch.float)
episode = []
for _ in range(episode_len):
    # generate (state, action, reward, next_state, next_action)
    state = next_state
    action = env.action_space.sample()
    # action_probs = agent.policy_net(state)
    # action = torch.max(action_probs, dim=-1).indices
    # choose action based on policy
    # m = torch.distributions.Categorical(action_probs)
    # action = m.sample()
    obs, reward, terminated , truncated, info = env.step(action)
    next_state = torch.tensor(obs, dtype = torch.float)
    episode.append(
        {
            "state": state,
            "action": action,
            "reward": reward,
            "next_state": next_state,
            # "action_probs": action_probs
        }
    )

"""
generate pytorch data iter
"""

rewards = []
states = []
actions = []
# action_probss = []
next_states = []
for i in range(len(episode)):
    rewards.append(episode[i]["reward"])
    actions.append(episode[i]["action"])
    # action_probss.append(episode[i]["action_probs"])
    states.append(episode[i]["state"])
    next_states.append(episode[i]["next_state"])
reward = torch.tensor(rewards, dtype=torch.float).reshape(-1, 1)
next_state = torch.stack(next_states)
action = torch.tensor(actions, dtype=torch.int).unsqueeze(1)
state = torch.stack(states)
# action_probs = torch.stack(action_probss)
data_arrays = (state, action, reward, next_state)
dataset = torch.utils.data.TensorDataset(*data_arrays)
data_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True, drop_last=False)




In [51]:
agent = A2CAgent(1, int(env.action_space.n), lr_policy=1e-5, lr_v=1e-5, discounted_factor=DISCOUNTED_FACTOR)
"""
execute algorithm        
"""

writer = SummaryWriter()
epochs = 600
iter_counter = torch.tensor(0.)
for episode in range(epochs):
        # next_action = torch.max(self.policy_net(next_state), dim=-1).indices
    total_reward = 0
    for state, action, reward, next_state in data_iter:

        # calculate advatage (TD-error)
        v_values = agent.v_net(state)
        TD_target = reward + agent.discounted_factor * agent.v_net(next_state)
        TD_error = TD_target - v_values

        TD_error = (TD_error - TD_error.mean()) / (TD_error.std() + 1e-8) # normalization
        # calculate probability of choosing this action if using current policy
        action_probs = agent.policy_net(state)
        # choose action based on policy
        m = torch.distributions.Categorical(action_probs)
        action = m.sample()


        # Actor: policy update
        action_prob = action_probs[torch.arange(len(action_probs)), action.squeeze()].unsqueeze(1)

        loss_actor = - TD_error * action_prob * agent.action_space # loss 或有问题
        # loss_actor = TD_error * - torch.log(action_prob) * (action_prob * agent.action_space) # loss 或有问题
        agent.optimizer_p.zero_grad()
        loss_actor.sum().backward(retain_graph=True) # 这里这样是否会出问题
        # loss_actor.sum().backward(inputs=list(agent.policy_net.parameters())) # 这里这样是否会出问题

        # torch.nn.utils.clip_grad.clip_grad_norm_(self.policy_net.parameters(), 100)
        # Critic: value update
        agent.optimizer_v.zero_grad()
        loss_critic = F.mse_loss(TD_target, v_values) * (action_prob * agent.action_space)
        # loss_critic = TD_error * - v_values * (action_prob * agent.action_space)
        loss_critic.sum().backward()
        # loss_critic.sum().backward(inputs=list(agent.v_net.parameters()))
        # torch.nn.utils.clip_grad.clip_grad_norm_(self.v_net.parameters(), 100)
        agent.optimizer_p.step()
        agent.optimizer_v.step()

        writer.add_scalar('TD_error', TD_error.sum(), iter_counter)
        writer.add_scalar('loss_critic', loss_critic.sum(), iter_counter)
        writer.add_scalar('loss_actor', loss_actor.sum(), iter_counter)
        total_reward += reward
        iter_counter += 1
    # writer.add_scalar('reward', total_reward.sum(), episode)
writer.flush()
writer.close()




RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x100 and 1x64)

 one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [100, 4]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead.

Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True)

In [ ]:

# policy = agent.generate_policy_table(env.height, env.width)
# print_by_dict(env, policy)
# print(env)
# for i in range(env.height):
#     print("[", end=" ")
#     for j in range(env.width):
#         state = (i,j)
#         action = np.argmax(policy[state])
#         print(env.action_mappings[action], end=" ")
#     print("]")

# V = agent.generate_v_table(env.height, env.width)
# print_by_dict(env, V)


[ [[0.24770188 0.25310567 0.25212058 0.24707192]] [[0.2529317  0.24682836 0.24636596 0.25387388]] [[0.2536649  0.25368395 0.2462884  0.24636272]] ]
[ [[0.24813594 0.25405413 0.24593219 0.25187778]] [[0.2507693  0.25126886 0.2462647  0.25169712]] [[0.24397117 0.2581954  0.24531654 0.2525169 ]] ]
[ [[0.24693333 0.25235036 0.25072017 0.2499962 ]] [[0.24877876 0.25035146 0.24969731 0.25117245]] [[0.2522654  0.2532033  0.24497296 0.2495583 ]] ]
[ 0.00 0.00 0.00 ]
[ 0.00 -10.00 0.00 ]
[ 0.00 0.00 1.00 ]

[  →   ←   →  ]
[  →   ←   →  ]
[  →   ←   →  ]
[ [[-27.86056]] [[-27.675497]] [[-27.999039]] ]
[ [[-27.980997]] [[-22.987865]] [[-28.114763]] ]
[ [[-28.340826]] [[-28.401997]] [[-28.892595]] ]


In [ ]:
# env = GridWorldEnv(render_mode="human", size=3,fixed_map = True, forbidden_grids=[(1,1)], target_grids=[(2,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
visualize_in_gym(agent, inp_env="CliffWalking-v0")
# gridworld_demo(agent, env)

reward: -1010, distance: [array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]), array([2, 2]),